In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

In [2]:
cut_off_date = '2023-08-5'

In [3]:
cut_off_date = datetime.strptime(cut_off_date, '%Y-%m-%d').date()

In [4]:
tms_codes = pd.read_excel('C:/Users/arun.lama/Desktop/GIMES1 WACC/New/FINAL checking file.xlsx')
tms_ids = tms_codes['tms_clean'].dropna().drop_duplicates()

In [54]:
data= pd.read_excel('C:/Users/arun.lama/Desktop/GIMES1 WACC/new/GIMES1 Floorsheet AFTER ADDING IPO DATA.xlsm')
data['CONTRACT_DATE'] = pd.to_datetime(data['CONTRACT_DATE'])

In [6]:
# tms_ids = ['0','20210408878']

In [7]:
all_info = []
for index, each_client in  enumerate(tms_ids):
    indi_data = data.copy()
    # 2021038995
    indi_data = indi_data[indi_data['TMS_CLIENT_CODE'] == each_client]
    indi_data.sort_values(by = ['CONTRACT_DATE'], ascending= True, inplace = True)
    floor_sht = indi_data
    indi_data['Quantity_buy_sell'] = np.where((indi_data['BUY_OR_SELL'] == "Sell"), indi_data['CONTRACT_QUANTITY'], -indi_data['CONTRACT_QUANTITY'])
    # ### If client has shares from IPO and buy only, no sell
    if set(indi_data['BUY_OR_SELL'].unique()).issubset(['Buy', 'IPO']) == True:
        indi_data['holding period'] = cut_off_date  - indi_data['CONTRACT_DATE'].dt.date
        indi_data['LongtermShortterm'] = np.where(indi_data['holding period'] > pd.Timedelta(days=365), 'Long Term', 'Short Term' )
        long_term = indi_data[indi_data['LongtermShortterm'] == 'Long Term']['CONTRACT_QUANTITY'].sum()
        short_term = indi_data[indi_data['LongtermShortterm'] == 'Short Term']['CONTRACT_QUANTITY'].sum()
        remarks = 'client has shares from IPO and buy only, no sell'
        condition = 1
        print('1',f'{each_client}, short term: {short_term}, long term: {long_term}')
    # ### If client has sell and IPO only, no buy
    elif  set(indi_data['BUY_OR_SELL'].unique()).issubset(['Sell', 'IPO']) == True:
        long_term = -indi_data['Quantity_buy_sell'].cumsum().iloc[-1]
        short_term = 0
        remarks = "client has sell and IPO only, no buy"
        print('2',f'{each_client}, short term: {short_term}, long term: {long_term}')
        condition = 1
    ### If client has/has not IPO, buy, and sell trades
    else:
        indi_data =  indi_data[[ 'ClientFullName', 'TMS_CLIENT_CODE', 'BUY_OR_SELL', 'CONTRACT_DATE','CONTRACT_QUANTITY', 'Quantity_buy_sell']]
        total_sell_qty = indi_data[indi_data['BUY_OR_SELL'] == 'Sell']['CONTRACT_QUANTITY'].sum()
        buy = indi_data[indi_data['BUY_OR_SELL'].isin(['Buy', 'IPO'])].copy()
        new_row = {'Quantity_buy_sell': total_sell_qty}
        buy.loc[-1] = new_row
        buy.index = buy.index + 1
        buy.sort_values(by = 'CONTRACT_DATE', na_position= 'first', inplace = True)
        buy['cum_sum_qty'] = buy['Quantity_buy_sell'].cumsum()
        remaining_stocks = buy[buy['cum_sum_qty']<0].copy()
        remaining_stocks['holding period'] = cut_off_date  - remaining_stocks['CONTRACT_DATE'].dt.date
        remaining_stocks['LongtermShortterm'] = np.where(remaining_stocks['holding period'] > pd.Timedelta(days=365), 'Long Term', 'Short Term' )
        if len(remaining_stocks) ==0 :
            long_term = 'error'
            short_term = 'error'
        else:
            if (remaining_stocks['LongtermShortterm'] == 'Long Term').all() == True:
                long_term = -remaining_stocks['cum_sum_qty'].iloc[-1]
                short_term = 0
            elif (remaining_stocks['LongtermShortterm'] == 'Short Term').all() == True:
                long_term = 0
                short_term =- remaining_stocks['cum_sum_qty'].iloc[-1]
            else:
                if len(remaining_stocks) == 2:
                    long_term = -remaining_stocks['Quantity_buy_sell'].iloc[0]
                    short_term = -remaining_stocks['Quantity_buy_sell'].iloc[1]
                else:
                    long_term = st = -remaining_stocks[remaining_stocks['LongtermShortterm'] == 'Long Term'].iloc[1:]['Quantity_buy_sell'].sum() - remaining_stocks['cum_sum_qty'].iloc[0] 
                    short_term = st = -remaining_stocks[remaining_stocks['LongtermShortterm'] == 'Short Term'].iloc[1:]['Quantity_buy_sell'].sum() - remaining_stocks['cum_sum_qty'].iloc[0]

        condition = 3
        print('3',f'{each_client}, short term: {short_term}, long term: {long_term}')
        
        
    remarks = "client has/does not IPO, buy, and sell trades"
    info = {'ClientID' : each_client,
        'Long Term' : long_term,
        'Short Term'     : short_term,
        'Remarks' : remarks
       }
    all_info.append(info)
    writer = pd.ExcelWriter(f'C:/Users/arun.lama/Desktop/GIMES1 WACC/New/HOLDINGS/{each_client}.xlsx', engine='openpyxl')
    
    if condition == 1:
        floor_sht.to_excel(writer, sheet_name='floorsheet', index=False)
    else:
        floor_sht.to_excel(writer, sheet_name='floorsheet', index=False)
        buy.to_excel(writer, sheet_name='FIFO Sales', index=False)
        remaining_stocks.to_excel(writer, sheet_name='Holdings', index=False)

    # Save and close the Excel writer for each loop iteration
    writer.save()
    writer.close()

In [8]:
final = pd.DataFrame.from_dict(all_info)
# final['Total'] = final['Long Term'] - final['Short Term']
final.to_csv('GIMES1 Holding Period python.csv')